# Trading Agent using Bayesian Networks

This notebook demonstrates how to build a trading agent using Bayesian Networks. It fetches stock market data, processes it, and uses Bayesian inference for decision-making.

In [ ]:
# Import libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
!pip install pgmpy
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

## Data Collection

In [ ]:
ticker = 'AAPL'
data = yf.download(ticker, start='2010-01-01', end='2023-12-31')
data.to_csv(f'{ticker}_data.csv')

## Data Exploration

In [ ]:
print(f'Number of observations: {len(data)}')
print('Data Description:')
print(data.describe())
data.info()
print('Missing Values:')
print(data.isnull().sum())

## Feature Engineering

In [ ]:
data['MA_20'] = data['Close'].rolling(20).mean()
data['RSI'] = 100 - (100 / (1 + data['Close'].diff(1).clip(lower=0).rolling(14).mean() /
                    data['Close'].diff(1).clip(upper=0).abs().rolling(14).mean()))
data['Volatility'] = data['Close'].pct_change().rolling(20).std() * np.sqrt(252)
data['Price_Movement'] = np.where(data['Close'].shift(-1) > data['Close'], 'Up', 'Down')
data.dropna(inplace=True)

## Discretization for Bayesian Network

In [ ]:
bins = {
    'RSI': [0, 30, 70, 100],
    'Volatility': [0, 0.2, 0.4, np.inf],
    'Volume': [0, data['Volume'].quantile(0.33), data['Volume'].quantile(0.66), np.inf]
}
labels = ['Low', 'Medium', 'High']
data['RSI_Cat'] = pd.cut(data['RSI'], bins=bins['RSI'], labels=labels)
data['Vol_Cat'] = pd.cut(data['Volatility'], bins=bins['Volatility'], labels=labels)
data['Vol_Chg'] = np.sign(data['Volume'].pct_change()).replace({1: 'Up', -1: 'Down', 0: 'No Change'}).fillna('No Change')

## Bayesian Network

In [ ]:
model = BayesianNetwork([
    ('RSI_Cat', 'Price_Movement'),
    ('Vol_Cat', 'Price_Movement'),
    ('Vol_Chg', 'Price_Movement'),
    ('Price_Movement', 'Action')
])
train_data = data.sample(frac=0.8, random_state=42)
test_data = data.drop(train_data.index)
train_data = train_data[['RSI_Cat', 'Vol_Cat', 'Vol_Chg', 'Price_Movement']]
model.fit(train_data, estimator=MaximumLikelihoodEstimator)

## Inference

In [ ]:
inference = VariableElimination(model)
query = inference.map_query(
    variables=['Action'],
    evidence={
        'RSI_Cat': 'Medium',
        'Vol_Cat': 'High',
        'Vol_Chg': 'Up'
    }
)
print(f'Recommended Action: {query['Action']}')